<a href="https://colab.research.google.com/github/Onoruoiza514/Playwright-automation-suite-/blob/main/aliexpress_clothes_scraping_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#We install dependencies
!pip install playwright
from playwright.async_api import async_playwright
import asyncio
# Install Playwright
#!pip install playwright

# Install the required browsers
!python3 -m playwright install

Playwright Host validation warning: 
╔══════════════════════════════════════════════════════╗
║ Host system is missing dependencies to run browsers. ║
║ Missing libraries:                                   ║
║     libwoff2dec.so.1.0.2                             ║
║     libgstgl-1.0.so.0                                ║
║     libgstcodecparsers-1.0.so.0                      ║
║     libavif.so.13                                    ║
║     libharfbuzz-icu.so.0                             ║
║     libenchant-2.so.2                                ║
║     libsecret-1.so.0                                 ║
║     libhyphen.so.0                                   ║
║     libmanette-0.2.so.0                              ║
╚══════════════════════════════════════════════════════╝
    at validateDependenciesLinux (/usr/local/lib/python3.10/dist-packages/playwright/driver/package/lib/server/registry/dependencies.js:216:9)
    at process.processTicksAndRejections (node:internal/process/task_queues:105

In [ ]:
import asyncio
import csv
from playwright.async_api import async_playwright

async def scrape_AliExpress():
    start_url = "https://www.aliexpress.com/category/100003109/women-clothing.html"

    async with async_playwright() as p:
        # Launch browser with timeout settings
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context()
        page = await context.new_page()

        # Set default timeout to 3 minutes
        page.set_default_timeout(180000)

        await page.goto(start_url)
        print("Navigated to AliExpress clothing store successfully")

        # Open CSV file for writing
        with open("scraped_ali_express_clothes_women_products.csv", "w", newline="", encoding="utf-8") as csvfile:
            fieldnames = ["id", "title", "price", "original_price", "sold", "store_name", "store_link", "product_link"]
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()

            product_id = 1

            async def scroll_to_load_more(page, selector):
                previous_count = 0
                while True:
                    await page.evaluate("window.scrollTo(0, document.body.scrollHeight)")
                    await asyncio.sleep(3)  # Wait for new content
                    current_count = await page.locator(selector).count()
                    if current_count == previous_count:
                        break
                    previous_count = current_count

            selectors = {
                "title": "h3[class=\"multi--titleText--nXeOvyr\"]",
                "price": 'div[class="multi--price-sale--U-S0jtj"]',
                "original_price": 'span[style="text-decoration: line-through; color: rgb(153, 153, 153);"]',
                "sold": 'span[class="multi--trade--Ktbl2jB"]',
                "store": 'span[class="cards--store--3GyJcot"] a',
                "product_link": 'a[class*="cards--storeLink--"]',
                "next_button": 'span[class="comet-icon comet-icon-arrowleftrtl32 "]'
            }

            while True:
                await scroll_to_load_more(page, selectors["title"])
                print("Finished scrolling on the current page")

                await page.wait_for_selector(selectors["title"], timeout=180000)

                product_titles = page.locator(selectors["title"])
                product_prices = page.locator(selectors["price"])
                product_original_prices = page.locator(selectors["original_price"])
                product_sold = page.locator(selectors["sold"])
                product_stores = page.locator(selectors["store"])
                product_links = page.locator(selectors["product_link"])

                product_count = await product_titles.count()
                for i in range(product_count):
                    try:
                        title = await product_titles.nth(i).inner_text()
                        price = await product_prices.nth(i).inner_text() if await product_prices.count() > i else "N/A"
                        original_price = await product_original_prices.nth(i).inner_text() if await product_original_prices.count() > i else "N/A"
                        sold = await product_sold.nth(i).inner_text() if await product_sold.count() > i else "N/A"
                        store_name = await product_stores.nth(i).inner_text() if await product_stores.count() > i else "N/A"
                        store_link = await product_stores.nth(i).get_attribute('href') if await product_stores.count() > i else "N/A"
                        product_link = await product_links.nth(i).get_attribute('href') if await product_links.count() > i else "N/A"

                        writer.writerow({
                            "id": product_id,
                            "title": title,
                            "price": price,
                            "original_price": original_price,
                            "sold": sold,
                            "store_name": store_name,
                            "store_link": store_link,
                            "product_link": product_link
                        })
                        print(f"Scraped product {product_id}: {title}")
                        product_id += 1
                    except Exception as e:
                        print(f"Error extracting details for product {product_id}: {e}")

                next_button = page.locator(selectors["next_button"])
                if await next_button.is_visible() and next_button.is_enabled():
                    print("Navigating to the next page...")
                    await next_button.first.click()
                    await asyncio.sleep(5)
                else:
                    print("No more pages to scrape")
                    break

        await browser.close()
        print(f"Scraping completed. Data saved to 'scraped_ali_express_clothes_women_products.csv'")

await scrape_AliExpress()


Navigated to AliExpress clothing store successfully
Finished scrolling on the current page
Scraped product 1: HXAO Women's Fur Coat Women 2024 Plush Burgundy Bomber Jacket Solid Crop Long Sleeve Jacket Warm Winter Coats Woman Casual Coats
Scraped product 2: TRAF 2024 Women Fashion Cropped Faux Fur Jacket Coat Long Sleeve Front Snap-button Female Outerwear Chic Lapel Collar Thick Coat
Scraped product 3: Women Retro Creative Hoodies Phoebe Bridgers Clothes Day After Tomorrow Music Album Sweatshirt Classic Scott ST Funny Pullovers
Scraped product 4: Spring Basic Women's T-shirts Half Turtleneck Slim Casual Pullover Ladies Tops Solid Modal Elastic Tees Long Sleeve For Women
Scraped product 5: TRAF Women's Formal Pants Office Wear Women Striped Grey Black Pants Woman High Waist Baggy Suit Pants Casual Wide Leg Trousers
Scraped product 6: Winter Sweet Plush Warm Hooded Sweet Coat New Japanese Fashion Princess White Cute Zipper Jacket Designer Slim Harajuku Clothing
Scraped product 7: Gray Bo

<ipython-input-14-7d96e54febe8>:88: RuntimeWarning: coroutine 'Locator.is_enabled' was never awaited
  if await next_button.is_visible() and next_button.is_enabled():


Finished scrolling on the current page
Scraped product 20: R2024 Harajuku Wide Leg Sweatpants for Women Elastic Waist Casual Sport Pants Woman Streetwear Gray Oversize Sportswear Female
Scraped product 21: Women's Black Gothic Baggy Jeans Y2K Japanese Harajuku Aesthetics Streetwear 2000s Retro Wide Legs Pants Jeans Clothing 2024
Scraped product 22: Y2K Fashion Jeans Men Women Harajuku Cherry Blossom Embroidery baggy Jeans Hip Hop StreetwearGothic Casual Wide Leg Denim Pants
Scraped product 23: 2025 Cement Grey Wide Leg Jeans Women's Fashion Four Seasons Slim Narrow Edition Cowboy Full Length Pants Female Brand Clothing
Scraped product 24: Y2K New Casual Joker Large Letter Embroidered Hoodie Men And Women High Street Couple Coat Retro Loose Zipper Sweatshirt Men
Scraped product 25: Retro Leopard Print Y2k Jeans for Women Harajuku Wide Leg Pants 2024 New Bottoms Denim Trousers Casual Straight Fashion Trousers
Scraped product 26: Fashion Trendy Printed Hoodies Funny Character Letters Grap

TimeoutError: Locator.click: Timeout 180000ms exceeded.
Call log:
  - waiting for locator("span[class=\"comet-icon comet-icon-arrowleftrtl32 \"]").first
  -     - locator resolved to <span class="comet-icon comet-icon-arrowleftrtl32 ">…</span>
  -   - attempting click action
  -     2 × waiting for element to be visible, enabled and stable
  -       - element is not enabled
  -     - retrying click action
  -     - waiting 20ms
  -     2 × waiting for element to be visible, enabled and stable
  -       - element is not enabled
  -     - retrying click action
  -       - waiting 100ms
  -     348 × waiting for element to be visible, enabled and stable
  -         - element is not enabled
  -       - retrying click action
  -         - waiting 500ms


In [ ]:
import pandas as pd
df = pd.read_csv("scraped_ali_express_clothes_women_products.csv")
print(df)

        id                                              title      price  \
0        1  HXAO Women's Fur Coat Women 2024 Plush Burgund...   MYR90.85   
1        2  TRAF 2024 Women Fashion Cropped Faux Fur Jacke...  MYR109.66   
2        3  Women Retro Creative Hoodies Phoebe Bridgers C...   MYR18.86   
3        4  Spring Basic Women's T-shirts Half Turtleneck ...    MYR64.6   
4        5  TRAF Women's Formal Pants Office Wear Women St...   MYR28.95   
...    ...                                                ...        ...   
1187  1188  Retro Flare Jeans Women American Style High Wa...   MYR83.87   
1188  1189  Wide Leg Pants For Women Palazzo Solid Color P...   MYR24.42   
1189  1190  Disney Mickey Minnie Mouse Y2k Hoodie Women's ...   MYR57.78   
1190  1191  Christmas Women's Disney Stitch Fashion Girls'...    MYR34.3   
1191  1192  Fine Imitation Wool Knitted T-Shirt Women's Sh...   MYR76.72   

     original_price         sold                store_name  \
0           MYR22.6  1,00

In [ ]:
#This is to print the description of theb first product scrapped
print(df['title'][1])

TRAF 2024 Women Fashion Cropped Faux Fur Jacket Coat Long Sleeve Front Snap-button Female Outerwear Chic Lapel Collar Thick Coat


In [ ]:
# Replace Null values NaN VALUES within the original price column with "MYR0"
df['original_price'] = df['original_price'].fillna("MYR0")

file_path = "scraped_ali_express_clothes_women_products.csv"


# Save the updated CSV file
df.to_csv(file_path, index=False)
print("Updated CSV file saved successfully!")
df

Updated CSV file saved successfully!


,id,title,price,original_price,sold,store_name,store_link,product_link
0,1,HXAO Women's Fur Coat Women 2024 Plush Burgund...,MYR90.85,MYR22.6,"1,000+ sold",HXAO Clothing Store,//www.aliexpress.com/store/1103663240,//www.aliexpress.com/store/1103663240
1,2,TRAF 2024 Women Fashion Cropped Faux Fur Jacke...,MYR109.66,MYR74.49,"5,000+ sold",TRAF-ZA Store,//www.aliexpress.com/store/1103069075,//www.aliexpress.com/store/1103069075
2,3,Women Retro Creative Hoodies Phoebe Bridgers C...,MYR18.86,MYR37.4,9 sold,B Dropshipping Store,//www.aliexpress.com/store/1101494097,//www.aliexpress.com/store/1101494097
3,4,Spring Basic Women's T-shirts Half Turtleneck ...,MYR64.6,MYR46.24,12 sold,KarSaNy Store,//www.aliexpress.com/store/1101319267,//www.aliexpress.com/store/1101319267
4,5,TRAF Women's Formal Pants Office Wear Women St...,MYR28.95,MYR165.61,"1,000+ sold",TRAF Vestidos Store,//www.aliexpress.com/store/1103377191,//www.aliexpress.com/store/1103377191
...,...,...,...,...,...,...,...,...
1187,1188,Retro Flare Jeans Women American Style High Wa...,MYR83.87,MYR0,17 sold,Sleeping Warm Store,//www.aliexpress.com/store/1101572912,//www.aliexpress.com/store/1101572912
1188,1189,Wide Leg Pants For Women Palazzo Solid Color P...,MYR24.42,MYR0,9 sold,MOLAVE Bolsos Store,//www.aliexpress.com/store/1101238212,//www.aliexpress.com/store/1101238212
1189,1190,Disney Mickey Minnie Mouse Y2k Hoodie Women's ...,MYR57.78,MYR0,7 sold,COOL DONG Store,//www.aliexpress.com/store/1103880779,//www.aliexpress.com/store/1103880779
1190,1191,Christmas Women's Disney Stitch Fashion Girls'...,MYR34.3,MYR0,119 sold,NEW 3D Store,//www.aliexpress.com/store/1103332258,//www.aliexpress.com/store/1103332258
